In [1]:
import os
import shutil
import numpy as np
import glob
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers
from tensorflow.keras.layers import Input, Add,Dropout, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import  plot_model
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

from IPython.display import SVG
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

In [2]:
### Mounting the google drive
from google.colab import drive
drive.mount('/content/datasets')

Mounted at /content/datasets


In [3]:
## unzip the dataset
!unzip "/content/datasets/MyDrive/archive.zip" > /dev/null

In [4]:
os.getcwd()

'/content'

In [17]:
# Where all dataset is there
data_dir = 'flowers'

# Training data dir
training_dir = 'data/Train'

# Test data dir
testing_dir = 'data/Test'

# Ratio of training and testing data
train_test_ratio = 0.8



def split_dataset_into_test_and_train_sets(all_data_dir = data_dir, training_data_dir = training_dir, testing_data_dir=testing_dir, train_test_ratio = 0.8):
    # Recreate testing and training directories
    if not os.path.exists('data'):
      os.mkdir('data')

    if not os.path.exists(training_data_dir):
      os.mkdir(training_data_dir)

    if not os.path.exists(testing_data_dir):
      os.mkdir(testing_data_dir)

    num_training_files = 0
    num_testing_files = 0


    for subdir, dirs, files in os.walk(all_data_dir):

        category_name = os.path.basename(subdir)

        # print(category_name + " vs " + os.path.basename(all_data_dir))
        if category_name == os.path.basename(all_data_dir):
              continue

        training_data_category_dir = training_data_dir + '/' + category_name
        testing_data_category_dir = testing_data_dir + '/' + category_name

        # creating subdir for each sub category
        if not os.path.exists(training_data_category_dir):
            os.mkdir(training_data_category_dir)

        if not os.path.exists(testing_data_category_dir):
            os.mkdir(testing_data_category_dir)

        file_list = glob.glob(os.path.join(subdir,'*.jpg'))

        #print(os.path.join(all_data_dir, subdir))
        print(str(category_name) + ' has ' + str(len(files)) + ' images')
        random_set = np.random.permutation((file_list))
        # copy percentage of data from each category to train and test directory
        train_list = random_set[:round(len(random_set)*(train_test_ratio))]
        test_list = random_set[-round(len(random_set)*(1-train_test_ratio)):]



        for lists in train_list :
            shutil.copy(lists, training_data_dir + '/' + category_name + '/' )
            num_training_files += 1

        for lists in test_list :
            shutil.copy(lists, testing_data_dir + '/' + category_name + '/' )
            num_testing_files += 1


    print("Processed " + str(num_training_files) + " training files.")
    print("Processed " + str(num_testing_files) + " testing files.")

In [19]:
split_dataset_into_test_and_train_sets()

dandelion has 1052 images
rose has 784 images
daisy has 764 images
sunflower has 733 images
tulip has 984 images
Processed 3453 training files.
Processed 864 testing files.


In [20]:
# Number of classes in dataset
num_classes = 5

def get_model():
    # Get base model
    # Here we are using ResNet50 as base model
    base_model = ResNet50(weights='imagenet', include_top=False)

    # As we are using ResNet model only for feature extraction and not adjusting the weights
    # we freeze the layers in base model
    for layer in base_model.layers:
        layer.trainable = False

    # Get base model output
    base_model_ouput = base_model.output

    # Adding our own layer
    x = GlobalAveragePooling2D()(base_model_ouput)
    # Adding fully connected layer
    x = Dense(512, activation='relu')(x)
    x = Dense(num_classes, activation='softmax', name='fcnew')(x)

    model = Model(inputs=base_model.input, outputs=x)
    return model

In [21]:
# Get the model
model = get_model()
# Compile it
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
# Summary of model
model.summary()

94765736/94765736 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, None, None, 3)        0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, None, None, 64)       9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, None, None, 64)       256       ['conv1_conv[0][0]']          
 on)                         

In [22]:
# Defining the imagedatagenerator for train and test image for pre-processing
# We don't give horizonal_flip or other preprocessing for validation data generator

image_size = 224
batch_size = 64

train_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input,
    shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
valid_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input)
train_generator = train_data_gen.flow_from_directory(training_dir, (image_size,image_size), batch_size=batch_size, class_mode='categorical')
valid_generator = valid_data_gen.flow_from_directory(testing_dir, (image_size,image_size), batch_size=batch_size, class_mode='categorical')


Found 3453 images belonging to 5 classes.
Found 864 images belonging to 5 classes.


In [23]:
# Training the fully conncected layer for initial epochs
epochs = 5

# Training the model

model.fit(
    train_generator,
    steps_per_epoch=train_generator.n//batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.n//batch_size,
    epochs=epochs,
    verbose=1)

Epoch 1/5
53/53 [==============================] - 58s 978ms/step - loss: 0.7281 - accuracy: 0.7359 - val_loss: 0.4360 - val_accuracy: 0.8498
Epoch 2/5
53/53 [==============================] - 52s 987ms/step - loss: 0.4133 - accuracy: 0.8625 - val_loss: 0.3562 - val_accuracy: 0.8798
Epoch 3/5
53/53 [==============================] - 52s 983ms/step - loss: 0.3583 - accuracy: 0.8817 - val_loss: 0.3191 - val_accuracy: 0.8966
Epoch 4/5
53/53 [==============================] - 50s 950ms/step - loss: 0.3067 - accuracy: 0.8961 - val_loss: 0.2949 - val_accuracy: 0.9062
Epoch 5/5
53/53 [==============================] - 51s 960ms/step - loss: 0.2818 - accuracy: 0.9059 - val_loss: 0.2771 - val_accuracy: 0.9087


In [26]:
# More fine tuning the model
# Training the model after 150 layers
# Generally ResNet is good at extracting lower level features so we are not fine tuning initial layers
epochs = 10

split_at = 140
for layer in model.layers[:split_at]: layer.trainable = False
for layer in model.layers[split_at:]: layer.trainable = True

model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

# Choosing lower learning rate for fine-tuning
# learning rate is generally 10-1000 times lower than normal learning rate, if we are fine tuning the initial layers
sgd = tf.keras.optimizers.legacy.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.n//batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.n//batch_size,
    epochs=epochs,
    verbose=1)

/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
<ipython-input-26-6307fd9b59c9>:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/10
53/53 [==============================] - 59s 1s/step - loss: 0.3718 - accuracy: 0.8846 - val_loss: 0.2597 - val_accuracy: 0.9123
Epoch 2/10
53/53 [==============================] - 51s 971ms/step - loss: 0.2392 - accuracy: 0.9209 - val_loss: 0.2251 - val_accuracy: 0.9219
Epoch 3/10
53/53 [==============================] - 51s 973ms/step - loss: 0.1910 - accuracy: 0.9407 - val_loss: 0.2098 - val_accuracy: 0.9243
Epoch 4/10
53/53 [==============================] - 52s 979ms/step - loss: 0.1446 - accuracy: 0.9608 - val_loss: 0.2042 - val_accuracy: 0.9303
Epoch 5/10
53/53 [==============================] - 50s 954ms/step - loss: 0.1188 - accuracy: 0.9670 - val_loss: 0.1969 - val_accuracy: 0.9279
Epoch 6/10
53/53 [==============================] - 50s 940ms/step - loss: 0.0950 - accuracy: 0.9726 - val_loss: 0.1934 - val_accuracy: 0.9339
Epoch 7/10
53/53 [==============================] - 51s 961ms/step - loss: 0.0780 - accuracy: 0.9799 - val_loss: 0.1796 - val_accuracy: 0.9387
Ep

In [27]:
print('Training complete')

Training complete
